In [31]:
import xarray as xr
import pandas as pd
import locale

# locale.setlocale(locale.LC_ALL,'en_US')
# locale.setlocale(locale.LC_ALL,'en_US')
dataset = xr.open_dataset('MiningProcess_Flotation_Plant_Database.h5', engine = 'h5netcdf')
df = dataset.to_pandas()


In [32]:
## Resampling
df = df.resample('1h').mean()

# # ##
# df.iloc[:-1, df.columns.get_loc('% Iron Feed')] = df['% Iron Feed'].iloc[1:].values
# df.iloc[:-1, df.columns.get_loc('% Silica Feed')] = df['% Silica Feed'].iloc[1:].values
# df.iloc[:-1, df.columns.get_loc('% Iron Concentrate')] = df['% Iron Concentrate'].iloc[1:].values
# df.iloc[:-1, df.columns.get_loc('% Silica Concentrate')] = df['% Silica Concentrate'].iloc[1:].values

# df = df.iloc[:-1]

## Split into input and output
X = df.iloc[:,:-2]
Y = df['% Silica Concentrate']

In [33]:
# Adding lagged variables

import numpy as np
X_copy = X.copy()
lagged_variables = {}
for lag in range(1,7):
    lagged_variable = np.zeros(len(X_copy))
    lagged_variable[lag:] = Y.iloc[:-lag]

    lagged_variables[f'% Silica Concentrate Lag {lag}'] = lagged_variable
    # X.loc[:,] = lagged_variable


for variable in X_copy.columns:
    X_variable = X_copy[variable]
    for lag in range(1,7):
        lagged_variable = np.zeros(len(X_variable))
        lagged_variable[lag:] = X_variable.iloc[:-lag]


        # X.loc[f'{variable} Lag {lag}'] = lagged_variable
        lagged_variables[f'{variable} Lag {lag}'] = lagged_variable

df_lagged_variables = pd.DataFrame(lagged_variables,index=X_copy.index)

X = X.join(df_lagged_variables)
X = X[5:]

In [34]:
remove_variables = True
N = 42
if remove_variables:
    variables = pd.read_csv('variables.csv',index_col=0)
    X = X[variables[:42]['Column']]

In [35]:
X

,% Silica Concentrate Lag 1,% Silica Concentrate Lag 4,Flotation Column 07 Level Lag 4,% Silica Concentrate Lag 3,% Iron Feed Lag 1,Ore Pulp Density Lag 5,% Silica Feed,% Silica Feed Lag 1,Starch Flow Lag 6,Flotation Column 03 Level Lag 2,...,Amina Flow Lag 5,% Silica Feed Lag 5,Flotation Column 03 Air Flow Lag 4,Starch Flow Lag 5,Flotation Column 03 Air Flow Lag 5,Amina Flow Lag 1,% Iron Feed Lag 4,Flotation Column 07 Level Lag 3,Flotation Column 07 Level Lag 5,Flotation Column 02 Air Flow Lag 4
date,,,,,,,,,,,,,,,,,,,,,
2017-03-10 06:00:00,1.34,1.11,451.387700,1.27,55.20,1.729558,16.98,16.98,0.000000,487.387206,...,578.786678,16.98,250.033317,3162.625026,250.178287,619.710806,55.20,450.296722,450.532747,250.214050
2017-03-10 07:00:00,1.15,1.27,450.296722,1.36,55.20,1.667784,17.40,16.98,3162.625026,549.459572,...,537.219661,16.98,250.046350,3133.256389,250.033317,621.878539,55.20,494.939889,451.387700,250.104167
2017-03-10 08:00:00,2.76,1.36,494.939889,1.34,54.95,1.732711,17.40,17.40,3133.256389,549.089244,...,591.906744,16.98,250.120861,3479.482944,250.046350,618.186367,55.20,550.271772,450.296722,250.204761
2017-03-10 09:00:00,2.65,1.34,550.271772,1.15,54.95,1.731056,17.40,17.40,3479.482944,550.995183,...,593.170106,16.98,250.013500,3228.036436,250.120861,595.680983,55.20,551.090767,494.939889,250.160494
2017-03-10 10:00:00,1.73,1.15,551.090767,2.76,54.95,1.765879,17.40,17.40,3228.036436,549.316628,...,619.710806,16.98,250.075383,3327.280739,250.013500,592.367983,55.20,550.575817,550.271772,250.111022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-09 19:00:00,1.83,2.46,401.485333,2.36,49.75,1.590255,23.20,23.20,2028.508887,400.018839,...,399.450422,12.90,299.919883,2257.033861,299.973961,489.346178,54.63,399.181494,400.916989,300.145544
2017-09-09 20:00:00,1.65,2.36,399.181494,2.74,49.75,1.685243,23.20,23.20,2257.033861,415.447022,...,575.549869,12.90,299.775617,2901.208994,299.919883,497.211189,49.75,401.583039,401.485333,300.276317
2017-09-09 21:00:00,1.71,2.74,401.583039,1.83,49.75,1.673219,23.20,23.20,2901.208994,403.858956,...,514.797428,23.20,299.950183,2783.602051,299.775617,508.963856,49.75,402.949094,399.181494,298.838408


In [36]:
# Split to calibration-validation set and test set
X_cal = X['20170615':'20170723']
Y_cal = Y['20170615':'20170723']
X_test = X['20170412':'20170506']
Y_test = Y['20170412':'20170506']

In [37]:
# Remove missing values in between
# X_cal.loc['201707060400':'201707062200'] = pd.NA
# X_cal.dropna(inplace=True)

# Y_cal.loc['201707060400':'201707062200'] = pd.NA
# Y_cal.dropna(inplace=True)

X_test.loc['20170421':'20170424'] = pd.NA
X_test.dropna(inplace=True)

Y_test.loc['20170421':'20170424'] = pd.NA
Y_test.dropna(inplace=True)

C:\Users\antto\AppData\Local\Temp\ipykernel_20280\695225842.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.dropna(inplace=True)


In [38]:
# Store data

with pd.HDFStore('MiningProcess_Flotation_Plant_Database_train_split.h5') as store:
    store.put('X_cal', X_cal)
    store.put('Y_cal', Y_cal)
    store.put('X_test', X_test)
    store.put('Y_test', Y_test)